# Cisco Blueprint 

# INE Notes 

## Overview 

### OSPF Resources
1. Anatomy of Internet protocol OSPF 
2. Routing TCP/IP vol 1
3. RFC 2328 OSPFv2
4. RFC 5340 OSPFv3

### What is OSPF
* Open standard IGP 
    * OSPFv2 : rfc 2328 
    * OSPFv3 : rfc 5340
* Link state protocol - used dijkstra's SPF algorithm 
* classelss protocol - supports VLSM and summarizaton. Ther are limitation on summarization point (ABR, ASBR) in order to maintain hierarchy in the LSRP.

### Why use OSPF ?
* __Gurantees loop free topology__ : all router builds exact same topology table, which then supplied into SPF, since every SPF gets same input the output is also deterministic thus, the loop free outcome is guranteed. 
* __Standard based__ : Interoperable between multiple vendors
* __Large Scalability__ : higheracical routing through Arias and topology summarization. 
* __Fast convergence__ : Actively track neighbours adjacencies and event driven incrimental updates 
* __Efficient Updates__ : Uses reliable multicast and unicast packets, non-ospf device don't need to process updates (unlike RIP that uses bradcast). 
* __Bandwidth based cost metric__ : better than hop count. 
* __Control plane Security__ : support multiple measures to be safe from MITM attacks e.g. MD5, SHA, IPSec etc. 
* __Extensible__ : future application support through "Opaque" LSA, e.g. MPLS traffic engineering 

### How OSPF Works ? 
1. discover OSPF neighbours and exchange topology information
2. choose best path via SPF
3. Neighbour and topology table maintainance 

### Step 1 : Neighbour and topology discovary 
* Like EIGRP, OSPF uses hello packets to discover neigbours on OSPF enabled attacjed links 
    * Transport via IP protocol number 89
    * sent as multicast 224.0.0.6 or 224.0.0.5 or unicast 
* Hello packets contains attributes that neighbours must agree on to form "Adjacency"
    * once negotiated, LSDB is exchanged. 
* Unique attributes 
    1. Router ID : node id in the link state graph, can be set manually or chosen automatically with the presidence of highest active loopback then highest active interface. 
    2. Interface IP address 
* Common Attribtes : Needs to match 
    1. Interface Area-ID 
    2. Hello and Dead interval 
    3. Interface Network ID
    4. MTU
    5. Network Type : Point-to-point, broadcast , point-to-multipoint 
    6. Authentication 
    7. Stub Flag
* OSPF Hello packet 
    * sent periodically using Hello_interval through OSPF enabled links 
* OSPF Adjacency state machine 
    1. __Down__ : No hellos have been received from the neighbour 
    2. __Attempt__ : 
        + Unicast hello packets have been sent but no hello is recived 
        + Only used for manually configured NBMA neighbours 
    3. __Init__ :
        + I have received a hello packet from a neighbour, but they have not acknowledged a hello from me
    4. __2 Way__ :
        + I have received a hello packet from my neighbour, and they have acknowledged a hello from me. 
        + Indicated by my RID in neighbours hello packets 
    5. __ExStart__ : 
        + First step to actual adjacencies 
        + master and slave relationship is formed, master has higher RID 
        + Master chooses the starting sequence number for the Database descriptor (DBD) packers that are used for actual LSA exchange 
    6. __Exchange__ :
        + local LSDB is sent through DBD packets 
        + DBD sequence number is used for reliable ack/re-trans 
    7. __Loading__:
        + Link-state request (LSR) packets are sent to ask for more information about a particular LSA 
    8. __Full__ :
        + Neighbours are fully adjacent and databases are synchonised. 
    
### Step 2: Choose Best path via SPF
* Once database is sync path selection begins 
* each router's LSA includes a cost attribute for each described link 
    - best path to that that link with lowest end-to-end cost 
    - Equal cost multi-path (ECMP) load balancing 
* bandwidth based cost 
    - $cost = \frac{\text{ref BW}}{\text{link BW}}$ , ref. BW =100M by default
    
### Step 3: Neighbour & Topology Maintainence 
* 